# 📦 Step 06: Elasticsearch에 벡터 데이터 저장

이 노트북에서는 이전 단계에서 생성한 벡터화된 문서들을 Elasticsearch에 저장하고 인덱스를 관리하는 방법을 알아봅니다.

## 📝 주요 내용
1. Elasticsearch 연결 설정
2. 벡터 검색을 위한 인덱스 매핑 설정
3. 벡터화된 문서 저장
4. 인덱스 상태 확인 및 관리

## 💡 Elasticsearch 벡터 검색 이해하기

Elasticsearch에서 벡터 검색을 위해서는 다음 사항들을 고려해야 합니다:

1. **인덱스 매핑**: dense_vector 타입을 사용하여 벡터 필드를 정의
2. **차원 설정**: OpenAI의 text-embedding-ada-002 모델은 1536 차원 벡터 생성
3. **유사도 메트릭**: cosine similarity를 사용하여 벡터 간 유사도 계산
4. **인덱스 설정**: 효율적인 검색을 위한 샤드 및 레플리카 구성

## 1️⃣ 필요한 라이브러리 임포트 및 환경 설정

Elasticsearch 연결에 필요한 라이브러리를 임포트하고 환경 변수를 설정합니다.

In [ ]:
import os
import json
from typing import List, Dict
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv
from tqdm.notebook import tqdm

# .env 파일에서 환경변수를 로드합니다
load_dotenv()

# Elasticsearch 클라이언트를 초기화합니다
es = Elasticsearch(
    hosts=[os.getenv('ELASTICSEARCH_URL', 'http://localhost:9200')],
    basic_auth=(
        os.getenv('ELASTICSEARCH_USERNAME', 'elastic'),
        os.getenv('ELASTICSEARCH_PASSWORD', '')
    )
)

# Elasticsearch 연결을 확인합니다
if es.ping():
    print("✅ Elasticsearch에 성공적으로 연결되었습니다.")
else:
    raise ConnectionError("❌ Elasticsearch 연결에 실패했습니다.")

## 2️⃣ 벡터 검색을 위한 인덱스 매핑 설정

Elasticsearch에서 벡터 검색을 위한 인덱스 매핑을 설정합니다.
- dense_vector 타입으로 임베딩 필드를 정의
- 문서 메타데이터를 위한 필드 추가
- 효율적인 검색을 위한 인덱스 설정

In [ ]:
# 인덱스 이름 설정
INDEX_NAME = "metadata-embeddings"

# 인덱스 매핑 정의
mapping = {
    "settings": {
        # 검색 성능 최적화를 위한 샤드 설정
        "number_of_shards": 1,
        "number_of_replicas": 1
    },
    "mappings": {
        "properties": {
            # 문서 원본 텍스트
            "content": {
                "type": "text",
                "analyzer": "standard"
            },
            # 문서 제목 또는 식별자
            "title": {
                "type": "keyword"
            },
            # 문서 종류 (테이블, 컬럼 등)
            "type": {
                "type": "keyword"
            },
            # OpenAI 임베딩 벡터 (1536 차원)
            "embedding": {
                "type": "dense_vector",
                "dims": 1536,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

# 기존 인덱스가 있다면 삭제
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
    print(f"🗑️ 기존 '{INDEX_NAME}' 인덱스를 삭제했습니다.")

# 새 인덱스 생성
es.indices.create(index=INDEX_NAME, body=mapping)
print(f"✨ 새로운 '{INDEX_NAME}' 인덱스를 생성했습니다.")

## 3️⃣ 벡터화된 문서 로드

이전 단계에서 생성한 벡터화된 문서들을 로드합니다.

In [ ]:
# 벡터화된 문서 파일 로드
with open('../data/processed/documents_with_embeddings.json', 'r', encoding='utf-8') as f:
    documents = json.load(f)

print(f"📄 로드된 문서 수: {len(documents)}")
print("\n문서 구조 예시:")
sample_doc = documents[0]
print(f"- 내용: {sample_doc['content'][:100]}...")
print(f"- 임베딩 차원: {len(sample_doc['embedding'])}")

## 4️⃣ Elasticsearch에 문서 저장

벡터화된 문서들을 Elasticsearch에 효율적으로 저장합니다.
- bulk API를 사용하여 대량 데이터 처리
- 진행 상황을 tqdm으로 시각화
- 에러 처리 및 결과 확인

In [ ]:
def generate_bulk_actions(documents: List[Dict]) -> List[Dict]:
    """
    Elasticsearch bulk API용 액션 리스트를 생성합니다.
    
    Args:
        documents: 벡터화된 문서 리스트
        
    Returns:
        bulk API 포맷의 액션 리스트
    """
    for i, doc in enumerate(documents):
        # 각 문서에 대한 인덱스 액션 정의
        yield {
            "_index": INDEX_NAME,
            "_id": str(i),  # 문서 ID
            "_source": {
                "content": doc["content"],
                "title": doc.get("title", f"문서_{i}"),
                "type": doc.get("type", "metadata"),
                "embedding": doc["embedding"]
            }
        }

# bulk API를 사용하여 문서 저장
try:
    with tqdm(total=len(documents), desc="문서 저장 중") as pbar:
        success, failed = bulk(
            es,
            generate_bulk_actions(documents),
            chunk_size=100,  # 한 번에 처리할 문서 수
            refresh=True,    # 저장 후 즉시 검색 가능하도록 설정
            raise_on_error=True
        )
        pbar.update(success)
    
    print(f"✅ 총 {success}개 문서가 성공적으로 저장되었습니다.")
    
except Exception as e:
    print(f"❌ 문서 저장 중 오류 발생: {str(e)}")

## 5️⃣ 인덱스 상태 확인

저장된 데이터의 상태와 인덱스 정보를 확인합니다.

In [ ]:
# 인덱스 상태 확인
index_stats = es.indices.stats(index=INDEX_NAME)
index_info = es.indices.get(index=INDEX_NAME)

print("📊 인덱스 상태:")
print(f"- 문서 수: {index_stats['_all']['total']['docs']['count']}")
print(f"- 저장 크기: {index_stats['_all']['total']['store']['size_in_bytes'] / 1024 / 1024:.2f} MB")

# 샘플 문서 검색으로 저장 확인
sample = es.search(
    index=INDEX_NAME,
    body={
        "size": 1,
        "query": {"match_all": {}}
    }
)

print("\n📄 저장된 문서 샘플:")
print(json.dumps(sample['hits']['hits'][0]['_source'], indent=2, ensure_ascii=False))

## 🎯 정리

이번 단계에서 완료한 작업:
1. Elasticsearch 연결 및 설정
2. 벡터 검색을 위한 인덱스 매핑 정의
3. 벡터화된 문서의 효율적인 저장
4. 인덱스 상태 확인 및 검증

다음 단계에서는 저장된 벡터 데이터를 사용하여 실제 검색을 수행하고
검색 결과의 품질을 평가해보겠습니다.